In [1]:
%load_ext dotenv
%dotenv -o

import os
os.environ["POLYGON_DATA_DIR"]

'/media/mirror/files.polygon.io'

In [2]:
from zipline_polygon_bundle import get_ticker_universe
from zipline_polygon_bundle.config import PolygonConfig

# from zipline_polygon_bundle import date_to_path, trades_schema, cast_trades, custom_aggs_schema, custom_aggs_partitioning, generate_csv_trades_tables, trades_to_custom_aggs
# from zipline_polygon_bundle import get_custom_aggs_dates

from typing import Iterator, Tuple

import pandas as pd
import plotly.graph_objs as go

import pyarrow as pa
from pyarrow import dataset as pa_ds
from pyarrow import compute as pa_compute
from pyarrow import compute as pc
from pyarrow import parquet as pa_parquet

from fsspec.implementations.arrow import ArrowFSWrapper

import os

import datetime
import pandas_market_calendars
import numpy as np

import resource

from itables import show

In [3]:
if not "CUSTOM_ASSET_FILES_DIR" in os.environ:
    os.environ["CUSTOM_ASSET_FILES_DIR"]= "/home/jovyan/data"

if not "POLYGON_TICKERS_DIR" in os.environ:
    os.environ["POLYGON_TICKERS_DIR"]= "/home/jovyan/data/tickers"

config = PolygonConfig(
    environ=os.environ,
    calendar_name="NYSE",
    start_date="2018-01-01",
    end_date="2023-12-31",
    agg_time="1min",
)

pd.set_option('display.width', 250)

# print(f"{config.minute_aggs_dir=}")
# print(f"{config.trades_dir=}")
print(f"{config.custom_aggs_dir=}")
# print(f"{config.aggs_dir=}")
# print(f"{config.by_ticker_dir=}")
print(f"{config.tickers_csv_path=}")

config.custom_aggs_dir='/home/jovyan/data/60sec_aggs'
config.tickers_csv_path='/home/jovyan/data/tickers/tickers_2018-01-01_2023-12-31.csv'


In [4]:
tickers = get_ticker_universe(config, fetch_missing=True)
tickers

tickers_csv_path='/home/jovyan/data/tickers/tickers_2018-01-01_2023-12-31.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49789 entries, ('CpP', 'XNYS', '0000831001', 'PFD', nan, True) to ('ZZrw', 'XNYS', '0000748015', 'RIGHT', nan, False)
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        49789 non-null  datetime64[ns]
 1   end_date          49789 non-null  datetime64[ns]
 2   last_updated_utc  49789 non-null  object        
 3   name              49789 non-null  object        
 4   share_class_figi  49789 non-null  object        
 5   delisted_utc      49789 non-null  object        
 6   currency_name     49789 non-null  object        
 7   locale            49789 non-null  object        
 8   market            49789 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 6.3+ MB


,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
CpP,XNYS,0000831001,PFD,NaN,True,2018-01-02,2018-01-02,2024-12-03,[Citigroup Inc],[],[],[usd],[us],[stocks]
XKII,ARCX,0001081019,NaN,BBG00JHNY9Z4,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Intelligent Structures ETF],[BBG00JHNYBP0],[],[usd],[us],[stocks]
XKST,ARCX,0001081019,NaN,BBG00JHP5PG6,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Smart Mobility ETF],[BBG00JHP5Q56],[],[usd],[us],[stocks]
LDRS,ARCX,0001415726,NaN,UNKNOWN,True,2018-01-02,2018-01-02,2024-12-03,[Innovator IBD� ETF Leaders ETF],[UNKNOWN],[],[usd],[us],[stocks]
XKFS,ARCX,0001081019,NaN,BBG00JHP6203,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Future Security ETF],[BBG00JHP62Q5],[],[usd],[us],[stocks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOEV,XNAS,0001750153,CS,BBG00NBK0GC3,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Class A Common Stock],[BBG00NBK0H68],[],[usd],[us],[stocks]
GOEVW,XNAS,0001750153,WARRANT,BBG00NWRF3R2,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Warrant],[],[],[usd],[us],[stocks]
GOEX,ARCX,0001432353,ETF,BBG0018HBWX2,True,2018-01-02,2023-12-29,2024-12-03,[Global X Gold Explorers ETF (NEW)],[BBG001TF8DG8],[],[usd],[us],[stocks]


In [5]:
tickers = get_ticker_universe(config, fetch_missing=True)
tickers

tickers_csv_path='/home/jovyan/data/tickers/tickers_2018-01-01_2023-12-31.csv'
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49789 entries, ('CpP', 'XNYS', '0000831001', 'PFD', nan, True) to ('ZZrw', 'XNYS', '0000748015', 'RIGHT', nan, False)
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_date        49789 non-null  datetime64[ns]
 1   end_date          49789 non-null  datetime64[ns]
 2   last_updated_utc  49789 non-null  object        
 3   name              49789 non-null  object        
 4   share_class_figi  49789 non-null  object        
 5   delisted_utc      49789 non-null  object        
 6   currency_name     49789 non-null  object        
 7   locale            49789 non-null  object        
 8   market            49789 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 6.3+ MB


,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
CpP,XNYS,0000831001,PFD,NaN,True,2018-01-02,2018-01-02,2024-12-03,[Citigroup Inc],[],[],[usd],[us],[stocks]
XKII,ARCX,0001081019,NaN,BBG00JHNY9Z4,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Intelligent Structures ETF],[BBG00JHNYBP0],[],[usd],[us],[stocks]
XKST,ARCX,0001081019,NaN,BBG00JHP5PG6,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Smart Mobility ETF],[BBG00JHP5Q56],[],[usd],[us],[stocks]
LDRS,ARCX,0001415726,NaN,UNKNOWN,True,2018-01-02,2018-01-02,2024-12-03,[Innovator IBD� ETF Leaders ETF],[UNKNOWN],[],[usd],[us],[stocks]
XKFS,ARCX,0001081019,NaN,BBG00JHP6203,True,2018-01-02,2018-01-02,2024-12-03,[SPDR Kensho Future Security ETF],[BBG00JHP62Q5],[],[usd],[us],[stocks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOEV,XNAS,0001750153,CS,BBG00NBK0GC3,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Class A Common Stock],[BBG00NBK0H68],[],[usd],[us],[stocks]
GOEVW,XNAS,0001750153,WARRANT,BBG00NWRF3R2,True,2020-12-22,2023-12-29,2024-12-03,[Canoo Inc. Warrant],[],[],[usd],[us],[stocks]
GOEX,ARCX,0001432353,ETF,BBG0018HBWX2,True,2018-01-02,2023-12-29,2024-12-03,[Global X Gold Explorers ETF (NEW)],[BBG001TF8DG8],[],[usd],[us],[stocks]


# Test Tickers
Need to use is_test from the Ticker Details JSON response because it isn't in their model class.

https://www.nasdaqtrader.com/Trader.aspx?id=SymbolDirDefs

https://github.com/polygon-io/client-python/blob/master/polygon/rest/models/tickers.py#L87

Also Ticker Details works for the base symbol but not always for the extensions:
This works:
https://api.polygon.io/v3/reference/tickers/ATEST.A?apiKey=$POLYGON_API_KEY
But not this:
https://api.polygon.io/v3/reference/tickers/NTEST.B?date=2022-10-19&apiKey=$POLYGON_API_KEY

Does this deal with test tickers?
https://alpaca.markets/sdks/python/api_reference/trading/models.html#alpaca.trading.models.Asset.tradable

In [6]:
[ticker for ticker in tickers.index.get_level_values('ticker').to_list() if "TEST" in ticker]

['NTEST.C',
 'NTEST.B',
 'NTEST.A',
 'NTEST',
 'ATEST',
 'ATEST.A',
 'ATEST.B',
 'ATEST.C']

In [7]:
tickers[tickers.index.get_level_values('ticker').str.contains("TEST")].sort_index()

,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,
ATEST,XASE,0001415311,NaN,BBG000Q2HK03,False,2018-01-02,2023-12-29,2016-10-25,[None],[BBG001S6X7W6],[2016-10-25],[usd],[us],[stocks]
ATEST.A,XASE,NaN,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
ATEST.B,XASE,0000849998,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
ATEST.C,XASE,0000860188,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST,XNYS,0001515980,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.A,XNYS,0001566897,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.B,XNYS,0000807884,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]
NTEST.C,XNYS,0000805264,NaN,NaN,False,2018-01-02,2023-12-29,2016-10-25,[None],[],[2016-10-25],[usd],[us],[stocks]


In [8]:
tickers[tickers.index.get_level_values('ticker').str.contains("ZZT")].sort_index()

,,,,,,start_date,end_date,last_updated_utc,name,share_class_figi,delisted_utc,currency_name,locale,market
ticker,primary_exchange,cik,type,composite_figi,active,,,,,,,,,


In [9]:
table = pa_parquet.read_table("/home/jovyan/data/60sec_aggs/year=2024/month=12/date=2024-12-23/part-0.parquet")
table

pyarrow.Table
ticker: string not null
window_start: timestamp[ns, tz=UTC]
open: double
high: double
low: double
close: double
total: double
volume: int64
transactions: int64
vwap: double
year: dictionary<values=int32, indices=int32, ordered=0>
month: dictionary<values=int32, indices=int32, ordered=0>
date: dictionary<values=string, indices=int32, ordered=0>
----
ticker: [["RUM","RUN","RZLV","SAIC","SBUX",...,"ZSL","ZTAX","ZTEST","ZTST","ZVOL"],["XLI","XLV","XMTR","XOM","XP",...,"TWM","TXN","U","UAVS","UBER"],...,["ACIU","ACIW","ACLS","ACLX","ACMR",...,"UNIT","UNL","UNM","UNP","UP"],["RBB","RBBN","RBC","RBLX","RBRK",...,"FFSM","FFTY","FFWM","FG","FGEN"]]
window_start: [[2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,2024-12-24 00:49:00.000000000Z,...,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z,2024-12-24 01:00:00.000000000Z],[2024-1

In [10]:
table.to_pandas()

,ticker,window_start,open,high,low,close,total,volume,transactions,vwap,year,month,date
0,RUM,2024-12-24 00:49:00+00:00,13.070,13.070,13.070,13.070,5881.50,450,2,13.070000,2024,12,2024-12-23
1,RUN,2024-12-24 00:49:00+00:00,10.030,10.030,10.030,10.030,5015.00,500,1,10.030000,2024,12,2024-12-23
2,RZLV,2024-12-24 00:49:00+00:00,3.270,3.270,3.270,3.270,7916.67,2421,2,3.270000,2024,12,2024-12-23
3,SAIC,2024-12-24 00:49:00+00:00,111.700,111.700,111.700,111.700,335.10,3,1,111.700000,2024,12,2024-12-23
4,SBUX,2024-12-24 00:49:00+00:00,87.490,87.490,87.490,87.490,1399.84,16,1,87.490000,2024,12,2024-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972232,FFSM,2024-12-23 19:58:00+00:00,26.920,26.920,26.920,26.920,134.60,5,2,26.920000,2024,12,2024-12-23
1972233,FFTY,2024-12-23 19:58:00+00:00,29.820,29.820,29.820,29.820,2982.00,100,2,29.820000,2024,12,2024-12-23
1972234,FFWM,2024-12-23 19:58:00+00:00,5.980,5.980,5.970,5.970,1289.68,216,3,5.970741,2024,12,2024-12-23
1972235,FG,2024-12-23 19:58:00+00:00,41.540,41.560,41.540,41.550,290.85,7,3,41.550000,2024,12,2024-12-23
